In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split

## Load Dataset

In [4]:
data = pd.read_excel("Data/Abusive Language Detection on Indonesian Online News Comments Dataset .xlsx")

In [5]:
data.head()

,Kalimat,label
0,san ente aje yg unboxing Indosat. masa ente ka...,1
1,"‚Å£‚Å£GOBLOK,ngapain beli indosat ? g ada untu...",3
2,Ngotot mau beli saham indosat kok jika jd pres...,1
3,Buyback Isat??anda sehat??sdh diminum obatnya?...,1
4,"Saya percaya kalau sama Sandiaga Uno, tapi kal...",1


In [6]:
data.shape

(3184, 2)

In [7]:
data['label'].value_counts()

1    2789
3     285
2     110
Name: label, dtype: int64

In [92]:
data[data['label']==3].sample(1)['Kalimat'].values[0]

'utekmu cuntellll hahahaha brand nempel dikaos dilarang wakaka lucu lucu..'

## Preprocess

In [23]:
prep_df = data[["Kalimat","label"]].copy()
prep_df = prep_df.reset_index()

In [24]:
#base
prep_df['Kalimat_prep'] = prep_df['Kalimat'].astype(str)
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.lower()

prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\n"," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\n"," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\r"," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\r"," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("[","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("]","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("[","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("  "," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("&amp", " ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("&gt", " ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("&lt", " ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\x[a-zA-z0-9][a-zA-z0-9]","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("'","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace('"','')
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("[0-9]","")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("[^A-Za-z]"," ")
prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace(r"\s+"," ")

prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.strip()
prep_df = prep_df.dropna(axis=0)
prep_df = prep_df[prep_df['Kalimat_prep']!=""]

C:\Users\rahma\AppData\Local\Temp\ipykernel_19308\4088594063.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\n"," ")
C:\Users\rahma\AppData\Local\Temp\ipykernel_19308\4088594063.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("\\\\r"," ")
C:\Users\rahma\AppData\Local\Temp\ipykernel_19308\4088594063.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].str.replace("[","")
C:\Users\rahma\AppData\Local\Temp\ipykernel_19308\4088594063.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single c

In [25]:
#acronym and typos
acro_typo = pd.read_excel("Data/Acronym Words and Typos .xlsx")

#singkatan dan kata dasar
singkatan = pd.read_excel("Data/kumpulan_singkatan_dan_kata_dasar.xls",header=None,names=['kata','kata sebenarnya'])

#normalize_dict
norm_dict = pd.concat([acro_typo,singkatan])

stdword = norm_dict['kata sebenarnya'].values.tolist()
nonstdword = norm_dict['kata'].values.tolist()

In [93]:
acro_typo.shape

(698, 2)

In [94]:
singkatan.shape

(214, 2)

In [99]:
acro_typo.head(10)

,kata,kata sebenarnya
0,kegiatn,kegiatan
1,trmasuk,termasuk
2,skalian,sekalian
3,hadehhh,hadeh
4,jelasssss,jelas
5,ajaa,aja
6,didlm,didalam
7,krna,karena
8,looo,lo
9,seh,sih


In [26]:
def normalize_text(text, stdword_, nonstdword_):
    text = text.split(" ")
    for i in range(len(text)):
        if text[i] in nonstdword_:
            index = nonstdword_.index(text[i])
            text[i] = stdword_[index]
    return ' '.join(map(str, text))

prep_df['Kalimat_prep'] = prep_df['Kalimat_prep'].apply(lambda x: normalize_text(x,stdword,nonstdword))

In [27]:
prep_df

,index,Kalimat,label,Kalimat_prep
0,0,san ente aje yg unboxing Indosat. masa ente ka...,1,santai kamu aja yang unboxing indosat masa kam...
1,1,"‚Å£‚Å£GOBLOK,ngapain beli indosat ? g ada untu...",3,goblok ngapain beli indosat tidak ada untungny...
2,2,Ngotot mau beli saham indosat kok jika jd pres...,1,ngotot mau beli saham indosat kok jika jadi pr...
3,3,Buyback Isat??anda sehat??sdh diminum obatnya?...,1,buyback isat anda sehat sudah diminum obatnya ...
4,4,"Saya percaya kalau sama Sandiaga Uno, tapi kal...",1,saya percaya kalau sama sandiaga uno tapi kala...
...,...,...,...,...
3179,3179,"hentikan aja, ngapai2n audisi2an, pantesa boca...",1,hentikan aja ngapain audisian pantesa boca kon...
3180,3180,Ini gerombolan orang2 yang mau meruntuhkan bib...,1,ini gerombolan orang yang mau meruntuhkan bibi...
3181,3181,KPAI isinya org bodoh ya.gmana pd djarum ga me...,3,kpai isinya org bodoh ya gimana padahal djarum...
3182,3182,Kpai idak ada gunanya. #bubarkan KPAI,1,kpai idak ada gunanya bubarkan kpai


## Train Test Split

In [28]:
# TRAIN TEST SPLIT
X = prep_df.drop("label",axis=1)
y = prep_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=8)

data_train = X_train.copy()
data_train['label'] = y_train
data_train = data_train.reset_index(drop=True)

data_test = X_test.copy()
data_test['label'] = y_test
data_test = data_test.reset_index(drop=True)

In [30]:
data_train.head()

,index,Kalimat,Kalimat_prep,label
0,986,Esia yang terlupakan \nGak kayak Smartfren sih...,esia yang terlupakan gak kayak smartfren sih y...,1
1,2670,"Kalau menurut saya, oknum KPAI yg cari panggun...",kalau menurut saya oknum kpai yang cari panggu...,1
2,1169,KUHP .. Kasih Uang Habis Perkara,kuhp kasih uang habis perkara,1
3,880,Kok boleh ngerekam? Bukannya xxi ada camera in...,kok boleh ngerekam bukannya xxi ada camera inf...,1
4,2701,Djarum kudus lgsg menghentikan bingung juga kp...,djarum kudus langsung menghentikan bingung jug...,1


In [31]:
data_test.head()

,index,Kalimat,Kalimat_prep,label
0,1269,justru KPAI sih yg harusnya sblm ngasih surat ...,justru kpai sih yang harusnya sebelum ngasih s...,1
1,2333,Jadi masih ada aturan lain yg lebih esensi utk...,jadi masih ada aturan lain yang lebih esensi u...,1
2,2419,pikiran KPAI kagak nyambung....pendek buanget....,pikiran kpai kagak nyambung pendek buanget kok...,1
3,158,Ngapain beli perusahaan yg mau ambruk dasar dungu,ngapain beli perusahaan yang mau ambruk dasar ...,1
4,2183,"Bukan lupa lagi judulnya, tapi blaga BUDEK iya...",bukan lupa lagi judulnya tapi blaga budek iya ...,3


In [32]:
prep_df.to_excel("Data/preprocessed_data.xlsx",index=False)
data_train.to_excel("Data/data_train.xlsx",index=False)
data_test.to_excel("Data/data_test.xlsx",index=False)